# 📊 Gerador de Data Schemas - GECOB

Este notebook executa o script para gerar automaticamente os data schemas das tabelas do GECOB.

**Tabelas documentadas:**
1. `gecob.prior_master_consolidado`
2. `gecob.prior_score_priorizacao`
3. `gecob.prior_score_componentes`
4. `gecob.prior_clusters_empresas`
5. `gecob.prior_outliers_identificados`

---

## 🔧 1. Setup do Ambiente

Configuração do Spark e imports necessários

In [ ]:
# Adicionar path das libs internas (se necessário)
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

# Imports PySpark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

# Imports libs internas
from utils import spark_utils_session as utils

print("✅ Imports realizados com sucesso")

## 🚀 2. Inicializar SparkSession

Criar ou obter a sessão Spark ativa

In [ ]:
# Configurar perfil e app name
profile = 'prod'  # ou 'dev', 'hom', conforme seu ambiente
app_name = "tsevero_gerar_data_schemas"

# Criar SparkSession
spark_builder = (utils.DBASparkAppSession
                 .builder
                 .setAppName(app_name)
                 .usingProcessProfile(profile)
                 .enableHiveSupport()
                 .enableHudiSupport())

# Build session
spark_session = spark_builder.build()
spark = spark_session.spark

print("✅ SparkSession criada com sucesso")
print(f"   App Name: {app_name}")
print(f"   Profile: {profile}")

## ✅ 3. Verificar Acesso às Tabelas

Teste rápido para confirmar acesso às tabelas

In [ ]:
# Testar acesso às tabelas
tabelas_teste = [
    'prior_master_consolidado',
    'prior_score_priorizacao',
    'prior_score_componentes',
    'prior_clusters_empresas',
    'prior_outliers_identificados'
]

print("🔍 Verificando acesso às tabelas...\n")

for tabela in tabelas_teste:
    try:
        count = spark.sql(f"SELECT COUNT(*) as total FROM gecob.{tabela}").collect()[0]['total']
        print(f"✅ {tabela:35s} - {count:,} registros")
    except Exception as e:
        print(f"❌ {tabela:35s} - ERRO: {str(e)[:50]}")

print("\n✅ Verificação concluída")

## 🎯 4. Executar Gerador de Schemas

Executa o script que gera todos os data schemas

In [ ]:
# Importar o script
import gerar_data_schemas

# Executar
gerar_data_schemas.main(spark)

## 📁 5. Verificar Arquivos Gerados

Listar os arquivos criados no diretório `data_schemas/`

In [ ]:
import os
import glob

output_dir = 'data_schemas'

if os.path.exists(output_dir):
    arquivos = sorted(glob.glob(f"{output_dir}/*"))
    
    print(f"📁 Arquivos gerados em '{output_dir}/':")
    print(f"   Total: {len(arquivos)} arquivos\n")
    
    # Agrupar por tipo
    tipos = {}
    for arquivo in arquivos:
        nome = os.path.basename(arquivo)
        ext = os.path.splitext(nome)[1]
        if ext not in tipos:
            tipos[ext] = []
        tipos[ext].append(nome)
    
    for ext, files in sorted(tipos.items()):
        print(f"\n{ext.upper() if ext else 'Outros':}:")
        for f in files:
            size = os.path.getsize(os.path.join(output_dir, f))
            size_kb = size / 1024
            print(f"   - {f:50s} ({size_kb:>8.1f} KB)")
else:
    print(f"❌ Diretório '{output_dir}/' não encontrado")

## 👁️ 6. Preview dos Resultados

Visualizar alguns exemplos dos schemas gerados

In [ ]:
import pandas as pd

# Exemplo: ver describe da tabela master
print("📊 DESCRIBE FORMATTED - prior_master_consolidado (primeiras 30 linhas)\n")

df = pd.read_csv('data_schemas/prior_master_consolidado_describe_formatted.csv')
display(df.head(30))

In [ ]:
# Exemplo: ver sample de dados
print("📋 SAMPLE DATA - prior_score_priorizacao\n")

df_sample = pd.read_csv('data_schemas/prior_score_priorizacao_sample_10.csv')
display(df_sample)

## 📄 7. Ver JSON Consolidado

Carregar e explorar o JSON consolidado com todos os schemas

In [ ]:
import json

# Carregar JSON consolidado
with open('data_schemas/data_schemas_completo.json', 'r', encoding='utf-8') as f:
    schemas = json.load(f)

print("📊 Resumo do JSON Consolidado:\n")

for schema in schemas:
    tabela = schema['tabela']
    row_count = schema.get('row_count', 'N/A')
    
    if 'error' not in schema:
        print(f"✅ {tabela:35s} - {row_count:>10,} registros")
    else:
        print(f"❌ {tabela:35s} - ERRO")

## 🧹 8. Cleanup (Opcional)

Encerrar sessão Spark

In [ ]:
# Descomentar para encerrar a sessão Spark
# spark.stop()
# print("✅ SparkSession encerrada")

---

## ✅ Processo Concluído!

Os data schemas foram gerados com sucesso. Você pode:

1. **Consultar o README**: `data_schemas/README.md`
2. **Ver schemas em Markdown**: `data_schemas/*_describe_formatted.md`
3. **Importar CSVs**: `data_schemas/*_describe_formatted.csv`
4. **Usar JSON**: `data_schemas/data_schemas_completo.json`

---

**GECOB - Sistema de Priorização de Cobrança**  
Receita Estadual de Santa Catarina